In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from typing_extensions import List, Dict, TypedDict, Annotated
from langgraph.graph import MessagesState

class State(MessagesState):
    product_name: str
    product_description: str
    category: str
    data: List[Dict]
    title: str
    bp: List[str]
    description: str

class TitleOutput(TypedDict):
    title: Annotated[str, ..., '규칙에 맞는 제목, 200자를 넘어가면 안됨']

class BPOutput(TypedDict):
    bp: Annotated[List[str], ..., '규칙에 맞는 BP, 각각 50자를 넘어가면 안됨']
    
class StateDescription(TypedDict):    
    description: Annotated[str, ..., '규칙에 맞는 설명, 2000자를 넘어가면 안됨']

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o', temperature=0)

In [ ]:
from langchain_core.prompts import PromptTemplate

title_template = """
다음 정보를 확인해서 아마존 상품의 타이틀을 작성해 주세요.
목표는 검색 노출을 극대화 하는 것으로, 반드시 아래 규칙을 지켜주세요

[규칙]
1. Title의 길이는 200자를 넘어가면 안됩니다
2. 첫 80자에는 핵심 내용(브랜드명, 제품명, Key attribute 등)이 들어가야 합니다
3. 후반부에는 제품과 관련된 중요한 키워드를 제시해야 합니다.

---
[데이터]
상품명: {product_name}
카테고리: {category}
상품 설명: {product_description}
---
[키워드]
{keyword}
---
[출력]
최종 타이틀:  
"""

title_prompt = PromptTemplate.from_template(title_template)


In [ ]:
def generate_title(state: State):
    prompt = title_prompt.invoke(
        {
            'product_name': state['product_name'], 
            'category': state['category'],
            'product_description': state['product_description'], 
            'keyword': state['data'],
        }
    )
    structured_llm = llm.with_structured_output(TitleOutput)
    res = structured_llm.invoke(prompt)
    return {'title': res['title']}

In [ ]:
from langgraph.graph import START, StateGraph

# 단순히 1열로 노드들이 진행되는 경우 만드는게 매우 간단함
builder = StateGraph(State).add_sequence(
    [generate_title]
)

builder.add_edge(START, 'generate_title')

graph = builder.compile()

from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
keywords = [
    {
        'keyword': 'chicken shredder',
        'rel_ct': '직접연관키워드',
        'qual_ct': '좋음'        
    },
    {
        'keyword': 'meat shredder claws',
        'rel_ct': '직접연관키워드',
        'qual_ct': '중간'        
    },
    {
        'keyword': 'meat grinder',
        'rel_ct': '중간연관키워드',
        'qual_ct': '중간'        
    },
    {
        'keyword': 'christmas gift kitchen',
        'rel_ct': '간접연관키워드',
        'qual_ct': '좋음'        
    },
    {
        'keyword': 'BBQ accessories',
        'rel_ct': '',
        'qual_ct': '낮음'        
    },
]

In [ ]:
final_state = graph.invoke({
    'product_name': 'Chicken Shredder',
    'category': 'Kitchen Gadgets',
    'product_description': '닭고기를 손쉽게 찢을 수 있는 도구입니다',
    'data': keywords,
})

final_state['title']